In [ ]:
import joblib
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm
import torch
import torch.nn as nn
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, Subset
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import KFold
import random

In [ ]:
loaded_data = joblib.load("train_data.pkl")
targets, data, ids = loaded_data

# 打印某个人的具体信息
for i in tqdm(range(277)):
    if ids[i] == "03d92c9f6f8a":
        print(f"Target: {targets[i]}")
        print(f"{data[i]}")
        print(type(targets[i]))
        print(type(data[i]))

for i in tqdm(range(277)):
    print(ids[i], end=" ")

In [ ]:
train_events = pd.read_csv("./train_events.csv")
num_person = len(train_events["series_id"].unique())
print("The number of total persons is:", num_person)

person_ids = train_events["series_id"].unique()
ids_to_index = {person_id: index for index, person_id in enumerate(person_ids)}
index_to_ids = {index: person_id for index, person_id in enumerate(person_ids)}
print(ids_to_index)
print(index_to_ids)

In [ ]:
class CFG:
    window_overlap = 36
    window_size = 18
    batch_size = 32
    input_dim = 2
    embed_dim = 32
    num_classes = 4 
    num_layers = 4
    nhead = 4
    dim_feedforward = 64
    learning_rate = 1e-3
    epochs = 300
    train_record_steps = 1
    dropout = 0.2
    num_person = 277
    scheduler_step_size = 30
    scheduler_gamma = 0.8
    fold = 8
    weight_decay = 0.001
    model_dir = "./models"

In [ ]:
#将每个人的数据分别提取出来

dic_X = {}
dic_y = {}

for i in tqdm(range(min(num_person, CFG.num_person))):

    print("Id:", ids[i])
    person_id = ids[i]

    dic_X[person_id] = []
    dic_y[person_id] = []

    pair_list = targets[i]
    onset_steps = []
    wakeup_steps = []
    for idx in range(len(pair_list)):
        onset_steps.append(pair_list[idx][0])
        wakeup_steps.append(pair_list[idx][1])

    empty_spaces = []
    events = train_events[train_events["series_id"] == person_id]
    last_events = 0
    empty_mark = 0 
    for (idx, row) in events.iterrows():
        if np.isnan(row["step"]):
            empty_mark = 1
        else:
            if row["step"] > last_events and empty_mark == 0:
                last_events = row["step"]
            elif row["step"] > last_events and empty_mark == 1:
                empty_spaces.append((int(last_events) - 360, int(row["step"]) - 360))
                last_events = row["step"]
                empty_mark = 0

    df = data[i]
    labels = []
    for j in range(0, len(df), CFG.window_overlap):
        start = j
        if j + CFG.window_size >= len(df):
            start = len(df) - CFG.window_size
            end = len(df)
            break
        else:
            end = j + CFG.window_size

        flag_empty = 0
        for k in empty_spaces:
            if (k[0] < start < k[1]) or (k[0] < end < k[1]):
                flag_empty = 1
                break
            else:
                continue

        flag_sleep = 0
        for k in range(len(onset_steps)):
            if (onset_steps[k] < start < end < wakeup_steps[k]):
                flag_sleep = 1
                break
            else:
                continue


        chunk = df.iloc[start:end][["anglez", "enmo"]]
        chunk = chunk.to_numpy()

        if flag_empty == 1:
            label = 4
        elif any(start <= num <= end for num in wakeup_steps):
            label = 3
        elif any(start <= num <= end for num in onset_steps):
            label = 2
        elif flag_sleep == 1:
            label = 1
        else:
            label = 0
        labels.append(label)

        if label != 4:
            dic_X[person_id].append(chunk)
            dic_y[person_id].append(label)

    element_counts = Counter(labels)

    # for element, count in element_counts.items():
    #     print(f"Element {element} occurs {count} times")

    print(len(dic_X[person_id]), len(dic_y[person_id]))

In [ ]:
for key, value in dic_y.items():
    y = value
    X = dic_X[key]

    X_awake = []
    y_awake = []
    X_sleep = []
    y_sleep = []
    X_onset = []
    y_onset = []
    X_wakeup = []
    y_wakeup = []

    for idx in range(len(y)):
        if y[idx] == 0:
            X_awake.append(X[idx])
            y_awake.append(y[idx])
        elif y[idx] == 1:
            X_sleep.append(X[idx])
            y_sleep.append(y[idx])
        elif y[idx] == 2:
            X_onset.append(X[idx])
            y_onset.append(y[idx])
        elif y[idx] == 3:
            X_wakeup.append(X[idx])
            y_wakeup.append(y[idx])

    num_sample = max(len(X_onset), len(X_wakeup))

    indices = random.sample(range(len(X_awake)), num_sample)
    selected_X_awake = [X_awake[i] for i in indices]
    selected_y_awake = [y_awake[i] for i in indices]
    X_awake = selected_X_awake
    y_awake = selected_y_awake

    indices = random.sample(range(len(X_sleep)), num_sample)
    selected_X_sleep = [X_sleep[i] for i in indices]
    selected_y_sleep = [y_sleep[i] for i in indices]
    X_sleep = selected_X_sleep
    y_sleep = selected_y_sleep

    # print(len(X_awake), len(X_sleep), len(X_onset), len(X_wakeup))

    X_total = []
    y_total = []
    X_total = X_awake + X_sleep + X_onset + X_wakeup
    y_total = y_awake + y_sleep + y_onset + y_wakeup

    dic_y[key] = y_total
    dic_X[key] = X_total

    print(len(dic_X[key]), len(dic_y[key]))

In [ ]:
class MyDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        inp = self.inputs[index]
        output = self.outputs[index]

        input_tensor = torch.tensor(inp, dtype=torch.float32)
        output_tensor = torch.tensor(output, dtype=torch.long)

        return input_tensor, output_tensor

In [ ]:
class TransformerModel(nn.Module):
    def __init__(
        self,
        input_dim,
        embed_dim,
        num_classes=4,
        num_layers=8,
        nhead=4,
        dim_feedforward=64,
        dropout=0.2
    ):

        super(TransformerModel, self).__init__()

        self.conv1d = nn.Conv1d(
            in_channels=input_dim, 
            out_channels=embed_dim, 
            kernel_size=3, 
            padding=1
        )
        self.embed_layer = nn.Linear(embed_dim, embed_dim)
        self.layernorm = nn.LayerNorm(embed_dim)

        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=embed_dim,
                nhead=nhead,
                dim_feedforward=dim_feedforward,
                dropout=dropout
            ),
            num_layers=num_layers
        )

        self.classification = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1d(x)
        x = x.permute(0, 2, 1)

        x = self.embed_layer(x)
        x = self.layernorm(x)
        x = self.encoder(x.permute(1, 0, 2))
        x = x[-1]
        x = self.classification(x)
        return x

# 示例
model = TransformerModel(input_dim=2, embed_dim=32)
input_data = torch.rand(3, 360, 2)
output = model(input_data)
print(model)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")
print(output.shape)

In [ ]:
model = TransformerModel(
    input_dim=CFG.input_dim,
    embed_dim=CFG.embed_dim,
    num_classes=CFG.num_classes,
    num_layers=CFG.num_layers,
    nhead=CFG.nhead,
    dim_feedforward=CFG.dim_feedforward,
    dropout=CFG.dropout
)

criterion = nn.CrossEntropyLoss()
device = "cuda" if torch.cuda.is_available() else "cpu"

total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

In [ ]:
train_series = pd.read_parquet("./train_series.parquet")

def inference(person_id, model):
    df = train_series[train_series["series_id"] == person_id]

    X = []
    y = []
    for j in range(0, len(df), CFG.window_overlap):
        start = j
        if j + CFG.window_size >= len(df):
            start = len(df) - CFG.window_size
            end = len(df)
            break
        else:
            end = j + CFG.window_size
            
        chunk = df.iloc[start:end][["anglez", "enmo"]]
        chunk = chunk.to_numpy()
        X.append(chunk)
        y.append(-1)

    dataset = MyDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=CFG.batch_size, shuffle=False)

    predictions = []
    
    for batch_idx, (data, target) in tqdm(enumerate(dataloader), total=len(dataloader)):

            data = data.to(device)
            target = target.to(device)
            output = model(data)
            _, predicted = torch.max(output, 1)
            print(predicted)
            predictions.extend(predicted.cpu().numpy())
            
    return predictions


In [ ]:
train_losses = []
train_step_losses = []
test_losses = []
learning_rates = []

indices = list(range(CFG.num_person))
kf = KFold(n_splits=CFG.fold, shuffle=True, random_state=2023)

for fold, (train_index, val_index) in enumerate(kf.split(indices)):
    # print(f"Fold {fold + 1}")
    print("Train indices:", train_index)
    print("Test indices:", val_index)
    X_train = []
    y_train = []
    for idx in train_index:
        person_id = index_to_ids[idx]
        X_train.extend(dic_X[person_id])
        y_train.extend(dic_y[person_id])

    X_val = []
    y_val = []
    for idx in val_index:
        person_id = index_to_ids[idx]
        X_val.extend(dic_X[person_id])
        y_val.extend(dic_y[person_id])
    
    print(len(X_train), len(X_val))

    train_dataset = MyDataset(X_train, y_train)
    val_dataset = MyDataset(X_val, y_val)
    train_dataloader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)

    train_model = model.to(device=device)
    train_model.train()

    optimizer = torch.optim.Adam(train_model.parameters(), lr=CFG.learning_rate)
    scheduler = StepLR(optimizer, step_size=CFG.scheduler_step_size, gamma=CFG.scheduler_gamma)

    min_loss = np.inf

    for epoch in range(1, CFG.epochs + 1):
        train_loss = 0.0
        test_loss = 0.0
        train_step_loss = 0.0

        learning_rate = optimizer.param_groups[0]["lr"]
        learning_rates.append(learning_rate)

        for batch_idx, (data, target) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
            optimizer.zero_grad()

            data = data.to(device)
            target = target.to(device)
            output = train_model(data)

            loss = criterion(output, target)
            train_loss += loss.item()
            train_step_loss += loss.item()

            loss.backward()
            optimizer.step()

            # if (batch_idx + 1) % CFG.train_record_steps == 0:
            #     train_step_losses.append(train_step_loss / (batch_idx + 1))
            #     print(f"Fold: {fold + 1}, Epoch: {epoch}, Step: {(batch_idx + 1)}, Train Loss Now: {train_step_loss / (batch_idx + 1)}")

        scheduler.step()

        with torch.no_grad():
            for batch_idx, (data, target) in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
                data = data.to(device)
                target = target.to(device)
                output = train_model(data)

                loss = criterion(output, target)
                test_loss += loss.item()

            for idx in val_index:
                person_id = index_to_ids[idx]
                predictions = inference(person_id=person_id, model=train_model)
                print(predictions)

        train_loss /= len(train_dataloader)
        train_losses.append(train_loss)

        test_loss /= len(val_dataloader)
        test_losses.append(test_loss)

        print(f"Fold: {fold + 1}, Epoch: {epoch}, Train Loss: {train_loss}, Test Loss: {test_loss}")

        if test_loss < min_loss:
            min_loss = test_loss
            torch.save(train_model.state_dict(), f"{CFG.model_dir}/model_epoch_{fold + 1}.pth")

In [ ]:
plt.figure(figsize=(20, 5))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label="Train Loss")
plt.plot(test_losses, label="Test Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Train & Test Loss")
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(learning_rates)
plt.xlabel("Epoch")
plt.ylabel("Learning Rate")

plt.savefig("plot.png")

plt.tight_layout()
plt.show()